In [ ]:
import os
import win32com.client

def excel_to_pdf_final_robust(excel_path, sheet_names, pdf_path):
    """
    通过调用Excel程序，将指定工作表高质量转换为PDF。
    此版本使用更健壮的 try/finally 结构来确保Excel进程被正确关闭。
    """
    excel = None
    workbook = None
    try:
        # --- 1. 初始化Excel并打开工作簿 ---
        excel = win32com.client.Dispatch("Excel.Application")
        excel.Visible = False
        excel.DisplayAlerts = False
        
        workbook = excel.Workbooks.Open(excel_path)

        # --- 2. 遍历每个工作表，进行页面设置 ---
        for sheet_name in sheet_names:
            print(f"正在为工作表 '{sheet_name}' 配置页面布局...")
            ws = workbook.Worksheets(sheet_name)
            
            ws.PageSetup.PrintArea = ws.UsedRange.Address
            
            # --- 这里是关键改动 ---
            # 直接使用数字 2 来代表横向，避免因环境问题找不到常量名
            ws.PageSetup.Orientation = 2  # 1 = 纵向(Portrait), 2 = 横向(Landscape)
            
            ws.PageSetup.Zoom = False
            ws.PageSetup.FitToPagesWide = 1
            ws.PageSetup.FitToPagesTall = 1
            
            ws.PageSetup.LeftMargin = excel.Application.InchesToPoints(0.5)
            ws.PageSetup.RightMargin = excel.Application.InchesToPoints(0.5)
            ws.PageSetup.TopMargin = excel.Application.InchesToPoints(0.5)
            ws.PageSetup.BottomMargin = excel.Application.InchesToPoints(0.5)

        # --- 3. 选择所有已配置好的工作表并导出 ---
        print(f"\n已完成所有页面配置，正在导出...")
        workbook.Worksheets(sheet_names).Select()
        
        # 参数 0 代表导出为PDF格式
        workbook.ActiveSheet.ExportAsFixedFormat(0, pdf_path)
        
        print(f"✅ 成功! PDF已保存至: {pdf_path}")

    except Exception as e:
        print(f"❌ 转换过程中发生错误: {e}")

    finally:
        # --- 4. 健壮的清理和退出过程 ---
        print("正在关闭资源...")
        if workbook:
            workbook.Close(SaveChanges=False)
        if excel:
            excel.Quit()
        
        workbook = None
        excel = None
        print("清理完成。")


# --- 配置参数 ---

# 1. Excel文件路径
excel_file_path = r'E:\excel.xlsx'
absolute_excel_path = os.path.abspath(excel_file_path)

# 2. 需要转换的sheet名称列表
sheets_to_convert = ['当月（同比）', '计划（同比）07', '累计（同比）']

# 3. 输出的PDF文件路径
output_pdf_path = os.path.splitext(absolute_excel_path)[0] + '.pdf'

# --- 运行转换 ---
excel_to_pdf_final_robust(absolute_excel_path, sheets_to_convert, output_pdf_path)

正在为工作表 '当月（同比）' 配置页面布局...
正在为工作表 '计划（同比）05' 配置页面布局...
正在为工作表 '累计（同比）' 配置页面布局...

已完成所有页面配置，正在导出...
✅ 成功! PDF已保存至: E:\2025年5月购电分析报表汇总因素分析.pdf
正在关闭资源...
清理完成。
